# Customer Segmentation

#### Setup

In [1]:
!pip install --upgrade --force-reinstall --no-deps kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 2.2 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=36a568bf7ac31baf91bed43b7f31fe03a984c569308f69822abb1550e09d8f23
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [2]:
import contextlib
from google.colab import files
import io

with contextlib.redirect_stdout(io.StringIO()):
    files.upload()

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

! kaggle datasets download frtgnn/dunnhumby-the-complete-journey

! unzip /content/dunnhumby-the-complete-journey.zip -d data

! rm -rf /content/dunnhumby-the-complete-journey.zip

## EDA

Let's look at the input files

In [ ]:
['/content/data/transaction_data.csv',
 '/content/data/campaign_table.csv',
 '/content/data/product.csv',
 '/content/data/coupon.csv',
 '/content/data/hh_demographic.csv',
 '/content/data/campaign_desc.csv',
 '/content/data/causal_data.csv',
 '/content/data/coupon_redempt.csv']

In [19]:
import os
import pandas as pd

trx = pd.read_csv('/content/data/transaction_data.csv')
trx.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [11]:
campaign = pd.read_csv('/content/data/campaign_table.csv')
campaign.head()

,DESCRIPTION,household_key,CAMPAIGN
0,TypeA,17,26
1,TypeA,27,26
2,TypeA,212,26
3,TypeA,208,26
4,TypeA,192,26


In [12]:
product = pd.read_csv('/content/data/product.csv')
product.head()

,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [13]:
coupons = pd.read_csv('/content/data/coupon.csv')
coupons.head()

,COUPON_UPC,PRODUCT_ID,CAMPAIGN
0,10000089061,27160,4
1,10000089064,27754,9
2,10000089073,28897,12
3,51800009050,28919,28
4,52100000076,28929,25


In [14]:
hh_demo = pd.read_csv('/content/data/hh_demographic.csv')
hh_demo.head()

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16


In [15]:
campaign_desc = pd.read_csv('/content/data/campaign_desc.csv')
campaign_desc.head()

,DESCRIPTION,CAMPAIGN,START_DAY,END_DAY
0,TypeB,24,659,719
1,TypeC,15,547,708
2,TypeB,25,659,691
3,TypeC,20,615,685
4,TypeB,23,646,684


In [16]:
causal_data = pd.read_csv('/content/data/causal_data.csv')
causal_data.head()

,PRODUCT_ID,STORE_ID,WEEK_NO,display,mailer
0,26190,286,70,0,A
1,26190,288,70,0,A
2,26190,289,70,0,A
3,26190,292,70,0,A
4,26190,293,70,0,A


In [17]:
coupon_redemp = pd.read_csv('/content/data/coupon_redempt.csv')
coupon_redemp.head()

,household_key,DAY,COUPON_UPC,CAMPAIGN
0,1,421,10000085364,8
1,1,421,51700010076,8
2,1,427,54200000033,8
3,1,597,10000085476,18
4,1,597,54200029176,18


### Adjusted Transational Data

With the raw data loaded, we need to make some adjustments to the transactional data. While this dataset is focused on retailer-managed campaigns, the inclusion of coupon discount matching information would indicate the transaction data reflects discounts originating from both retailer- and manufacturer-generated coupons. Without the ability to link a specific product-transaction to a specific coupon (when a redemption takes place), we will assume that <u>any coupon_discount value associated with a non-zero coupon_discount_match value originates from a manufacturer's coupon. All other coupon discounts will be assumed to be from retailer-generated coupons.</u>

In addition to the separation of retailer and manufacturer coupon discounts, we will calculate a <u>list amount for a product as the sales amount minus all discounts applied</u>:

In [21]:
trx[trx['COUPON_MATCH_DISC'] != 0].head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
134,718,26985360571,1,855325,1,1.10,324,-0.69,1115,1,-0.40,-0.40
154,718,26985360571,1,948756,3,2.12,324,-0.90,1115,1,-0.75,-0.25
210,718,26985360571,1,9368449,1,2.05,324,-0.49,1115,1,-0.55,-0.45
236,2305,26996870743,2,1043128,2,1.53,414,-1.00,1300,1,-0.55,-0.45
594,122,27008905909,3,6514251,3,4.25,330,-1.87,1156,1,-2.25,-0.75


In [22]:
trx['AMOUNT_LIST'] = trx['SALES_VALUE'] - trx['RETAIL_DISC'] - trx['COUPON_DISC'] - trx['COUPON_MATCH_DISC']

In [25]:
# CASE 
#         WHEN COALESCE(coupon_discount_match,0.0) != 0.0 THEN -1 * COALESCE(coupon_discount,0.0) 
#         ELSE 0.0 
#         END as manuf_coupon_discount,

def create_discount(x, retailer = 'Y'):
  cpn_mtch_disc = x['COUPON_MATCH_DISC']
  cpn_disc = x['COUPON_DISC']

  if retailer == 'Y':
    if (cpn_mtch_disc == 0) or (cpn_mtch_disc != cpn_mtch_disc):
      return -1 * cpn_disc
    else:
      return 0
  else:
    if (cpn_mtch_disc != 0) and (cpn_mtch_disc != cpn_mtch_disc):
      return -1 * cpn_disc
    else:
      return 0


trx['CAMPAIGN_COUPON_DISC'] = trx.apply(lambda x : create_discount(x, 'Y'), axis = 1)
trx['MANUF_COUPON_DISC'] = trx.apply(lambda x : create_discount(x, 'N'), axis = 1)